<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong>
        Importando bibliotecas
    </strong>
</p>

In [1]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import numbers

import xlwings as xw


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Lendo base de dados
    </strong>
</p>

In [2]:
def ler_arquivo(nome_arquivo):
    try:
       return pd.read_excel(f'bases/{nome_arquivo}.xlsx')
    except FileNotFoundError as e:
        return f'Erro: {e}'


<p style="font-size: 1.1rem; color: #9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Dados ausentes        
    </strong>
</p>

In [3]:
df = ler_arquivo('dados de entregas')


<p style="font-size: 1.1rem; color: #9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        SLA    
    </strong>
</p>

In [4]:
#total de entregas
df_total_entregas = df.loc[ df['status'] == 'ENTREGUE']
df_total_entregas

#Total de entregas sem atraso
sla_ok = df_total_entregas.loc[ df['dias_atraso'] == 0].shape[0]
sla_ok

#Percentual SLA
sla_percentual = (sla_ok / df_total_entregas.shape[0]) * 100
f'{round(sla_percentual,2)}% das entregas foram feitas dentro do prazo.'

'37.24% das entregas foram feitas dentro do prazo.'

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        KPI - Entragas atrasadas
    </strong>
</p>

In [5]:
# Total de entregas
total_entregas = df_total_entregas.shape[0]

# Total de entregas com atraso
entregas_atraso = df_total_entregas.loc[df_total_entregas['dias_atraso'] > 0].shape[0]

# Percentual de atraso
percentual_atraso = (entregas_atraso / total_entregas) * 100

print(f"{round(percentual_atraso, 1)}% das entregas tiveram atraso.")


62.8% das entregas tiveram atraso.


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Tipagem correta para ID
    </strong>
</p>

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Padronização e Normalização
    </strong>
</p>

Padronização de descrição

In [6]:
df['descricao_entrega'] = (
    df['descricao_entrega']
    .str.lower()
    .str.strip()
)

Separar as entregas atrasadas


In [8]:
df_atrasos = df.loc[df['dias_atraso']>0].copy()


Categorizando as causas dos atrasos

In [12]:
df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains("chuva|temporal|tempestade",na=False),
    "causa_atraso"
] = 'Clima'


df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains("operacional|rota|extravio",na=False),
    "causa_atraso"
] = 'Problema Operacional'

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains("reagendada|cliente",na=False),
    "causa_atraso"
] = 'Entrega reagendada'

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains("normal",na=False),
    "causa_atraso"
] = 'entregue'

df_atrasos['causa_atraso'] = df_atrasos['causa_atraso'].fillna("Outros")

In [ ]:
df['descricao_entrega'].head()

Pirncipais causas de atrasos 

In [14]:
round(df_atrasos["causa_atraso"].value_counts(normalize=True) * 100, 1).reset_index(name="%")

,causa_atraso,%
0,Problema Operacional,25.1
1,Entrega reagendada,24.8
2,Outros,24.7
3,entregue,12.8
4,Clima,12.7


Atrasos por filiais

In [17]:
df_atrasos.groupby(['filial','causa_atraso']).size() \
            .reset_index(name='Quantidade') \
            .sort_values(['filial','Quantidade'],ascending=[True, False])


,filial,causa_atraso,Quantidade
1,MG,Entrega reagendada,146
2,MG,Outros,138
3,MG,Problema Operacional,137
0,MG,Clima,84
4,MG,entregue,73
6,PR,Entrega reagendada,157
7,PR,Outros,154
8,PR,Problema Operacional,153
5,PR,Clima,86
9,PR,entregue,84
